# Dashboard Panels

<img src='http://panel.pyviz.org/_static/logo_stacked.png' width=20% align='right'>

Panel makes it simple to make:

* Plots with user-defined controls
* Property sheets for editing parameters of objects in a workflow
* Control panels for simulations or experiments
* Custom data-exploration tools
* Dashboards reporting key performance indicators (KPIs) and trends
* Data-rich Python-backed web servers
* and anything in between

There are three key components to a Panel:

* `Pane` objects allow wrapping external viewable items like Bokeh, Plotly, Vega, or HoloViews plots, so they can be embedded in a panel.
* `Widget` objects provide controls that can trigger Python or JavaScript events.
* `Panel` layout objects allow combining plots into a Row, Column, Tabs or a Grid.

# Table of Contents
* [Dashboard Panels](#Dashboard-Panels)
	* [Setup](#Setup)
	* [Pane types](#Pane-types)
	* [Rows and Columns](#Rows-and-Columns)
	* [Widgets](#Widgets)
	* [Full Dashboard](#Full-Dashboard)
	* [Deployments](#Deployments)


## Setup

In [ ]:
import matplotlib
matplotlib.use('agg')

import pandas as pd
import hvplot.pandas
import panel as pn
pn.extension()

In [ ]:
auto = pd.read_csv('auto-mpg.csv')

## Pane types

Panel also allows text, images, plots, and other objects to be composed into a visualization..

In [ ]:
title = pn.panel('# Plots of the `auto-mpg` data set', width=700)
title

In [ ]:
image = pn.panel('http://panel.pyviz.org/_static/logo_stacked.png', width=80)
image

## Rows and Columns

To construct the full visualization, *Panes* are composed into *Rows* and *Columns*.

In [ ]:
top = pn.Row(title, image)
top

HoloViews (hvplot) objects are automatically converted into Panes when included in Rows and Columns. For Matplotlib objets it is best to use the figure itself, rather than the axes.

In [ ]:
scatter1 = auto.hvplot.scatter(x='hp', y='mpg', by='origin', width=500, height=300)

kde = auto['mpg'].plot.kde().get_figure()

In [ ]:
plots = pn.Row(kde, scatter1)

The full panel is a collection of Rows and columns.

In [ ]:
panel = pn.Column(top, plots)
panel

Here's the complete specification as a single column of two Rows.

```python
panel = pn.Column(
    pn.Row(title, image),
    pn.Row(kde, scatter1)
)
```

## Widgets

Instead of using the `@interact` decorator, which automatically generates widgets, they can be constructed and included separately in *Rows* and *Columns*.

See the [Widget documentation](https://panel.pyviz.org/user_guide/Widgets.html) for more options.

In [ ]:
min_year = int(auto['yr'].min())
max_year = int(auto['yr'].max())
increment = 1

year = pn.widgets.IntSlider(name='Year',
                            start=min_year,
                            end=max_year,
                            step=increment,
                            value=min_year,
                            width=800)

metric = pn.widgets.RadioBoxGroup(name='Metric',
                                  options=['mpg','displ','hp','weight','accel'])

In [ ]:
@pn.depends(year, metric)
def average(year, metric):
    idx = auto['yr'] == year
    values = auto[metric]
    return auto.loc[idx, metric].hvplot.kde(width=800, xlim=(values.min(), values.max()))

average_pane = pn.Column(
    year, pn.Row(average, metric)
)
average_pane

## Full Dashboard

Panes allow composition of the entire dashboard including widgets, text, images, etc. The `+` operation is best used when you want the plots to *share tools*. In this case the X ranges are linked.

In [ ]:
plot_opts = {
    'by':'origin',
    'hover_cols': ['name','yr','cyl'],
    'width':400,
}

scatter1 = auto.hvplot.scatter(x='hp', y='mpg',     legend='top_right',    **plot_opts)
scatter2 = auto.hvplot.scatter(x='hp', y='weight',  legend='bottom_right', **plot_opts)
scatter = scatter1 + scatter2
scatter

In [ ]:
dashboard = pn.Column(
    top, scatter, average_pane
)
dashboard

## Deployments

Any *Pane*, *Row*, or *Column* object can be served as a dashboard from within the Notebook or script file with the member method `.servable()`. This indicates to the Panel Server that only those plots will appear in the deployment.

In [ ]:
dashboard.servable();